In [ ]:
%load_ext autoreload
%autoreload 2

Perform analysis of the parquet exported data using a Dask cluster.

In [ ]:
import dask
import dask.array as da
import dask.dataframe as dd
import dask.distributed
import dask_jobqueue
import numpy as np
import os
import pathlib
import pandas as pd

In [ ]:
DATA_DIR = pathlib.Path(os.getenv('DATA_DIR'))

# Boot cluster

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ~/.bash_profile','conda activate smc01'],
    name='smc01-dask',
)

In [ ]:
cluster.scale(jobs=2)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

# Read dataset

In [ ]:
INPUT_DIR = DATA_DIR / '2021-03-17-ppdataset/'

In [ ]:
df = dd.read_parquet(DATA_DIR / '2021-03-17-ppdataset/*.parquet')

In [ ]:
df

In [ ]:
df = df.persist()

In [ ]:
(df['date'] == '2019-06-01').sum().compute()

In [ ]:
subset = df[df['date'] == '2019-06-01'].compute()

In [ ]:
df['error_2t'] = df['gdps_2t'] - df['obs_2t']
df['squared_error_2t'] = (df['gdps_2t'] - df['obs_2t']) ** 2
df['step_hour'] = df['step'] / 3600
df['step_td'] = dd.to_timedelta(df['step'], unit='S')
df['valid'] = df['date'] + df['step_td']

In [ ]:
error_by_step = df.groupby('step_hour').mean()['squared_error_2t'].compute()

In [ ]:
error_by_step.plot()

In [ ]:
error_by_run = df.groupby('date').mean()['squared_error_2t'].compute()

In [ ]:
np.sqrt(error_by_run).plot()

In [ ]:
obs_counts = df['station'].value_counts().compute()

In [ ]:
obs_counts

In [ ]:
obs_counts > 30

In [ ]:
type(obs_counts)

In [ ]:
stations_with_obs = obs_counts.index[obs_counts > 30].tolist()

In [ ]:
subset = df[df['station'].isin(stations_with_obs)]

In [ ]:
subset

In [ ]:
subset['station'].value_counts().compute().min()

In [ ]:
subset = subset.categorize(columns=['station'])

In [ ]:
subset = subset.persist()

In [ ]:
station_ids = subset['station'].cat.codes.values

In [ ]:
station_ids

In [ ]:
feature_columns = [c for c in subset.columns if c.startswith('gdps')]

In [ ]:
feature_columns

In [ ]:
features = subset[feature_columns]

In [ ]:
features.mean().compute()

In [ ]:
features.std().compute()

In [ ]:
one_hot_station = dd.get_dummies(subset['station'])

In [ ]:
one_hot_station

In [ ]:
subset['valid']

In [ ]:
subset['yearly_component'] = da.sin((subset['valid'].dt.dayofyear / 366) * 2*np.pi )

In [ ]:
subset['daily_component'] = da.sin((subset['valid'].dt.hour / 24) * 2*np.pi)

In [ ]:
subset['step_component'] = (subset['step'] / (237 * 60 * 60))

In [ ]:
temporal_features = subset[['yearly_component', 'daily_component', 'step_component']]

In [ ]:
temporal_features.describe().compute()

In [ ]:
temporal_features_array = temporal_features.to_dask_array()

In [ ]:
temporal_features_array

In [ ]:
station_ids

In [ ]:
features_array = da.concatenate([temporal_features_array, features.to_dask_array()], axis=1, allow_unknown_chunksizes=True)

In [ ]:
features_array.compute_chunk_sizes()

In [ ]:
train_mask = (subset['date'].dt.year == 2019).values

In [ ]:
val_mask = (subset['date'].dt.year == 2020).values

In [ ]:
train_mask.compute_chunk_sizes()

In [ ]:
val_mask.compute_chunk_sizes()

In [ ]:
train_features = features_array[train_mask]

In [ ]:
val_features = features_array[val_mask]

In [ ]:
train_features.compute_chunk_sizes()
train_features = train_features.rechunk()

In [ ]:
train_features

In [ ]:
train_features_comp = train_features.compute()

In [ ]:
train_features_comp.to_hdf5(DATA_DIR / '2021-03-17-ppdataset/test.hdf', '/train/x')

In [ ]:
station_ids_comp = station_ids.compute()

In [ ]:
import h5py

In [ ]:
f = h5py.File(DATA_DIR / '2021-03-17-ppdataset/test.hdf', 'w')

In [ ]:
f.create_dataset('/train/x/', shape=train_features_comp.shape, dtype=train_features_comp.dtype, data=train_features_comp, compression='lzf')

In [ ]:
def array_to_hdf(file, name, array):
    file.create_dataset(name, shape=array.shape, dtype=array.dtype, data=array)

In [ ]:
f.create_dataset('/train/stations', shape=train_features_comp.shape, dtype=train_features_comp.dtype, data=train_features_comp, compression='lzf')

In [ ]:
array_to_hdf(f, '/train/stations', station_ids_comp)

In [ ]:
train_y = subset['error_2t'].to_dask_array()

In [ ]:
train_y.compute_chunk_sizes()

In [ ]:
train_y_comput = train_y[train_mask].compute()

In [ ]:
array_to_hdf(f, '/train/y', train_y_comput)

In [ ]:
val_x_compute = val_features.compute()

In [ ]:
val_mask.compute_chunk_size()

In [ ]:
station_ids.compute_chunk_sizes()

In [ ]:
val_stations = station_ids[val_mask]

In [ ]:
array_to_hdf(f, '/val/x', )

In [ ]:
f.close()